In [1]:
!pip install transformers

!pip install -U sentence-transformers




     |████████████████████████████████| 3.8 MB 5.5 MB/s 
     |████████████████████████████████| 895 kB 26.0 MB/s 
     |████████████████████████████████| 6.6 MB 31.0 MB/s 
     |████████████████████████████████| 67 kB 2.9 MB/s 
     |████████████████████████████████| 596 kB 42.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 79 kB 3.6 MB/s 
     |████████████████████████████████| 1.2 MB 28.8 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=a5bc792d867ee93768d327d7ed7de51c5e8f22a07981dde6cf315e6caa894b51
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers


In [2]:
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import numpy as np
from os.path import exists

import re
from numpy.core.fromnumeric import argmax

In [3]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline

model_name = "deepset/roberta-base-squad2"


In [4]:
# a) Get predictions
nlp = pipeline('question-answering', model=model_name, tokenizer=model_name)

model = SentenceTransformer('sentence-transformers/paraphrase-xlm-r-multilingual-v1')

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/473M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/550 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [17]:
if exists("responses.txt"):
  response_file = open("responses.txt", "r")
  responses = response_file.read()
  response_file.close()    
else:
  responses="""It's not an easy situation. She has good days and bad days. And on the bad days, I get affected the most. You know my son helps a lot around the house but he is working on his PhD. and he doesn’t have time to help me out that much. It's just very painful to watch it deteriorate and the drinking helps me cope.
  My wife's at home with Alzheimer's. It's not an easy situation. My wife has good days and bad days. And on the bad days, I get affected the most. You know my son helps a lot around the house but he is working on his PhD. and he doesn’t have time to help me out that much. It's just very painful to watch it deteriorate and the drinking helps me cope.
  My wife's at home with Alzheimer's. It's a difficult scenario. There are good days and bad days for my wife. And it's on terrible days that I'm the most affected. You know how much my son helps around the house, but he is working on his PhD and doesn't have much time to assist me. Watching it decay is excruciating, and drinking helps me cope.
  My wife's at home with Alzheimer's. It's a challenging situation. For my wife, there are good days and bad days. And it's the bad days that have the most impact on me. You know how much my son helps around the house, but he is working on his PhD and doesn't have a lot of spare time to help me. Things's awful to see it deteriorate, and drinking helps me cope.
  Hi I am Pastor Zimmerman.
  Well. I've been pastoring the church for the last 25 years and I have built up a nice congregation. My wife's at home with Alzheimer's. I feel very guilty these days because I've been drinking pretty heavily and I don't want to disappoint people. But it’s the only way I’ve been able to cope lately.
  Well. I've been pastoring the church for the past 25 years, and I've amassed quite a following. My wife has Alzheimer's disease and is confined to her home. I'm feeling a lot of guilt these days because I've been drinking a lot and don't want to disappoint anyone. But it's the only way I've been able to deal with things recently.

  Well. I've been pastoring the church for the past 25 years, and I've amassed quite a following. My wife has Alzheimer's disease and is confined to her house. I'm feeling a lot of guilt these days since I've been drinking a lot and don't want to disappoint anyone. But it's the only way I've been able to deal with things recently.


  Yes that is correct
  Yes that is right


  Yes , at least an eight.
  Yes, atleast an eight out of 10

  I am visiting because I had a nodule removed from my throat
  I am here because I had a nodule removal from my throat

  I drink as a coping mechanism to numb my daily life problems with my wife
  I drink to cope with my problems regarding my wife


  One thing is what happened to my throat. I could also lose face with those who have placed a lot of trust in me. Overall, it is detrimental to my health.
  When I focus on trying to exercise more, when I meet with friends but I don't have a lot of time to do that with my current situation. That’s why I could never get any consistent with any of that.
  When I attempt to exercise more and meet up with friends, but I don't have a lot of time to do so because of my current circumstances. That's why I've never been able to keep up with any of it.
  When I try to exercise more and meet up with friends, but due to my current circumstances, I don't have a lot of time to do so. I've never been able to keep up with any of it because of this.
  When I attempt to exercise more and meet up with pals, but I don't have a lot of time owing to my current circumstances. Because of this, I've never been able to keep up with anything.

  You know I've been successful sometimes. I can cut back to two or three beers a day but then something will always come up to trigger it again. You know something will happen at church, I have a problem with trying to communicate with my wife or something in the family.
  Yes sometimes. I can cut back to two or three beers a day but then something will always come up to trigger it again. You know something will happen at church, I have a problem with trying to communicate with my wife or something in the family.
  Well I had a nodule removed from my throat and I am facing some drinking issues due to family problems
  I got a nodule removed from my throat and I am drinking a lot nowadays due to family problems
  Yes I can try that
  That's a great suggestion, I can try that
  Thank you so much for meeting with me, have a great day.

  Two or three beers a night with a couple of shots.

  about two to three bottles.
  Hi, nice to meet you
  Hi, thanks for meeting me
  Hi I am branden kay and I will help you
  Hi pastor zimmerman, I am branden kay and I will be helping you today
  Hi Pastor Zimmerman, I am Branden Kay and I will be assisting you today
  Hi, I'm Branden Kay
  Hi pastor. I'm . I am a nursing practitioner student. I will be talking with you today.
  Hi pastor Zimmerman. I'm Branden Kay. I am a nurse practitioner student. I will be working with you today.

  Back in the day I used to swim every day. That was very relaxing. But I can't take advantage of it. You know with my current situation if I could just find some way to do it regularly, I think it will help.
  I used to swim every day when I was younger. That was really soothing. However, I am unable to take use of it. With my current position, I believe that if I could only find a method to do it on a regular basis, it would be beneficial.
  When I was younger, I used to swim every day. That was really relaxing. I, on the other hand, am unable to make use of it. With my current situation, I feel it would be good if I could only find a way to do it on a daily basis.
  Yes that would be great 
  Yeah sure.

  I am drinking a lot

  I have drinking problems


  That would be great.
  Yes that would be really nice, thank you
  That would be really great, thank you

  Hello!
  Good day!
  That’s probably about six or seven. It’s because I haven’t been successful in maintaining it in the past.
  That's maybe six or seven. It's because I've never been able to keep it up in the past.

  That would be around six or seven. It's because in the past, I've never been able to keep up with it.
  My son is working on his PhD.

  My son is doing great. He is working on his PhD.

  He helps around the house a bit and he is working on his PhD.

  I didn't get that. Can you say it again?
  I missed what you said. What was that?
  No I haven't. That's a good idea. I know I can join Alcohols and Anonymous. Sometimes my wife's situation could be beneficial.
  No, I haven't done so. That's an excellent suggestion. I am aware that I am eligible to join Alcoholics Anonymous. My wife's condition can be advantageous at times.
  I haven't done anything like that. That is a wonderful idea. I'm conscious that I'm a candidate for membership in Alcoholics Anonymous. At times, my wife's condition is beneficial.
  No, I haven't done anything like that. That is a fantastic suggestion. I am aware that I am a candidate for membership in Alcoholics Anonymous. My wife's condition can be advantageous at times.
  Sure, no problem
  Yes Sure
  Yes, no problem
  Yes glad to do so
  Yes, that might be a good way of trying to wean myself off the heavy stuff.
  Yes that is a good idea to drink only beer
  It calms me down, helps me sleep, and makes it easier for me to communicate with people.
  I know it's not healthy for me but it's the most cost-efficient way for me to calm down these days.
  It calms me down


  I got the nodule because I was drinking too much.
  """

In [18]:
if exists("intents.txt"):
  intents_file = open("intents.txt", "r")
  intents = intents_file.read()
  intents_file.close()    
else:  
  intents="""
  <break>
  so you are using alcohol as a coping mechanism?
  Sounds difficult. It seems that you are using alcohol as a coping mechanism.
  That sounds very hard. You may be using alcohol as a coping mechanism. 
  That sounds difficult and it seems like you are using alcohol as a coping mechanism.
  That sounds incredibly hard. And it sounds like you are using alcohol to cope with it.
  That sounds like it would be incredibly hard. So it sounds like you are using alcohol as a coping mechanism.
  <break>

  Yes, there is nothing I am proud of.
  Yes but I am not proud of it
  Yes but I am not happy about it



  <break>

  alcohol abuse benefits?
  alcohol benefits?
  drinking benifits?
  benefits of drinking?
  Name some benefits of drinking?
  So a score of 16 means you are drinking at a risky level. However it makes sense considering you are using it to cope with your home life. I want to give you feedback on your drinking. There are good and bad things to your drinking. Are you able to name some of the benefits you get?
  You scored a 16 which means you are drinking at risk levels. I understand it is something that you are using to cope with your home life. But I wanted to give you feedback on that.There are pros and cons to your drinking. Can you give me some benefits you get?

  A score of 16 means that you are drinking at a risky level, but I understand that it is what you use to cope with various events at home. But I just wanted to give you some feedback on that. Obviously there are good and bad things about your drinking. Are you able to name some of the benefits you get?
  pros of drinking?
  What are the advantages of drinking?
  Why do you drink?
  Advantages of drinking?
  What are the pros of drinking?
  Okay you scored a 16 which means you are drinking it at risk levels and I understand it is something that you are using to cope with a lot going on at home. But I just wanted to give you some feedback on that. Obviously there are good and bad things about your drinking. Are you able to name some of the benefits you get?
  <break>


  It calms me down, helps me sleep, and makes it easier for me to communicate with people.
  I know it's not healthy for me but it's the most cost-efficient way for me to calm down these days.
  It calms me down
  <break>

  only beer try?
  Please try only beer?
  Only beer drinking?
  Can you try to use only beer?
  <break>

  Yes, that might be a good way of trying to wean myself off the heavy stuff.
  Yes that is a good idea to drink only beer

  <break>
  alcohol use assessment go through okay?
  Is it okay to go over your alcohol use assessment?
  You have completed the audit, an alcohol use assessment. We have given it to all of our patients. Is it fine if I go over it with you?
  You have completed the audit, an alcohol use assessment. We have given it to all of our patients. Is it ok if I go over it with you?
  You have completed the audit, an alcohol use assessment. We give it to all of our patients. Is it ok if I go over it with you?
  You completed the audit, which is an alcohol use assessment. We give it to all of our patients. Is it ok if I go over it with you?
  Now, today, you have completed an audit of your alcohol use assessment. It's been given to all the patients. Do you mind if I review it with you?
  Okay, you completed the audit, it's an alcohol use assessment today. We give it to all of our patients. Is it ok if I go over it with you?
  <break>


  Sure, no problem
  Yes Sure
  Yes, no problem
  Yes glad to do so
  <break>

  Alzheimer's support group?
  Alzheimer's support group?
  a circle that could help you with what you are going through with your wife?
  Have you thought of an Alzheimer's support group?
  It seems that when you use other coping mechanisms, you are more successful in cutting back on your alcohol use. However, it seems that you have alot going on with your wife. Have you thought of an Alzheimer's support group? Or a circle that could help you with what you are going through with your wife?

  Hm, I believe that when you use other coping mechanisms, you are more successful in cutting back on your alcohol use. However, it seems that you have alot going on with your wife. Have you thought of an Alzheimer's support group? Or a circle that could help you with what you are going through with your wife?

  Sounds like when you use other coping mechanisms you've been more successful in cutting back on your alcohol use. Right now, however, you have a lot going on with your wife. Have you thought about an Alzheimer's support group or something that could help you have a circle that could help you with what you are dealing with your wife?

  It sounds to me like when you use other coping mechanisms you've been more successful in cutting back on your alcohol use. But right now you have a lot going on with your wife.   Have you thought about an Alzheimer's support group or something that could help you have a circle that could help you with what you are dealing with your wife?

  <break>

  No I haven't. That's a good idea. I know I can join Alcohols and Anonymous. Sometimes my wife's situation could be beneficial.
  No, I haven't done so. That's an excellent suggestion. I am aware that I am eligible to join Alcoholics Anonymous. My wife's condition can be advantageous at times.
  I haven't done anything like that. That is a wonderful idea. I'm conscious that I'm a candidate for membership in Alcoholics Anonymous. At times, my wife's condition is beneficial.
  No, I haven't done anything like that. That is a fantastic suggestion. I am aware that I am a candidate for membership in Alcoholics Anonymous. My wife's condition can be advantageous at times.
  <break>


  How confident are you that you'd be able to do that?
  So how confident are you that you'd be able to do that?
  Now, how confident are you that you would be able to do that?
  How confident are you that you would be able to do that?
  <break>


  That’s probably about six or seven. It’s because I haven’t been successful in maintaining it in the past.
  That's maybe six or seven. It's because I've never been able to keep it up in the past.

  That would be around six or seven. It's because in the past, I've never been able to keep up with it.


  I didn't get that. Can you say it again?
  I missed what you said. What was that?
  <break>
  just going to say hi
  heya
  hello hi
  howdy
  hey there
  long time no see
  <break>

  Hello!
  Good day!

  <break>



  would you like to come up with a solution together?
  Maybe we can come up with some solutions together. Would you be ok with talking about that?
  We could come up with some solutions together. Would you like that?
  Hm, it seems like there is some difference between your willingness and confidence. We could come up with some solutions together. Would you like that?
  It sounds like there is some difference between your willingness and confidence. We could come up with some solutions together. Is that fine with you?
  Okay. It sounds like there's a little bit of discrepancy between your willingness and your confidence.  Maybe we can come up with some solutions together. Would you be ok with talking about that?
  Okay. so it sounds like there is a little bit of discrepancy between your willingness and your confidence.  Maybe we can come up with some solutions together. Would you be ok with talking about that?
  <break>

  That would be great.
  Yes that would be really nice, thank you
  That would be really great, thank you

  <break>

  Maybe we could talk more about this the next time we meet. Would that be okay?
  Should we discuss this on your next visit?
  Can we discuss this on your next visit?
  Some in-home exercise may help relieve some of your stress. It sounds like you are on-board with this idea. Can we discuss this on your next visit?
  In-home exercise could help relieve some of your stress. It sounds like you are willing to work with this plan. Can we discuss this on your next visit?
  Some in-home exercise may help relieve some of your stress. It sounds like you are willing to work with this plan. Can we discuss this on your next visit?
  Yes, maybe even some in-home exercise would help a bit with your stress relief.  It sounds like you are willing to work on this. Maybe I can talk to you about this again at the next visit?
  Yes, maybe even some in-home exercise would help a little bit with your stress relief.  It sounds like you are willing to work on this. Maybe I can talk to you about this again at the next visit?

  <break>

  Yes that would be great 
  Yeah sure.
  <break>


  exercise?
  How about some exercise?
  Could you drink two 12-ounces a night at first to cut back and then see how that goes for you? How about some exercise? Has that worked for you in the past to help deal with the stress?
  Can you drink two 12-ounces a night at first to cut back and then see how that goes? What about some exercise? Did it work for you in the past to help with your stress?
  So maybe if we can drink two 12-ounces a night at first to cut back and then slowly see how that goes? How about some exercise? Did it work for you in the past to help with your stress?

  So maybe if we can drink two 12-ounces a night at first to cut back and then slowly see how that goes? What about some exercise? Did it work for you in the past to help with your stress?

  <break>

  Back in the day I used to swim every day. That was very relaxing. But I can't take advantage of it. You know with my current situation if I could just find some way to do it regularly, I think it will help.
  I used to swim every day when I was younger. That was really soothing. However, I am unable to take use of it. With my current position, I believe that if I could only find a method to do it on a regular basis, it would be beneficial.
  When I was younger, I used to swim every day. That was really relaxing. I, on the other hand, am unable to make use of it. With my current situation, I feel it would be good if I could only find a way to do it on a daily basis.
  <break>


  hi I am branden kay and help you today I

  Hi I am branden kay and I will help you
  Hi pastor zimmerman, I am branden kay and I will be helping you today
  Hi Pastor Zimmerman, I am Branden Kay and I will be assisting you today
  Hi, I'm Branden Kay
  Hi pastor. I'm . I am a nursing practitioner student. I will be talking with you today.
  Hi pastor Zimmerman. I'm Branden Kay. I am a nurse practitioner student. I will be working with you today.

  Hi pastor Zimmerman. I am . I am a nursing student. I will be talking with you today.
  <break>


  Hi, nice to meet you
  Hi, thanks for meeting me

  <break>
  How much beer do you drink?
  How much beer are you drinking?
  <break>

  about two to three bottles.
  <break>

  How much drinking?
  How much do you normally drink?
  Well instead of cutting out alcohol completely, you could cut back a few drinks. How much do you normally drink?
  Instead of cutting out alcohol completely, you could cut back at first. How much alcohol do you normally drink?
  Okay, so maybe instead of cutting out alcohol altogether you could cut back at first. How much do you drink?
  Okay, so maybe instead of cutting out alcohol altogether you could cut back at first. How much are you drinking when you are drinking alcohol

  <break>
  Two or three beers a night with a couple of shots.

  <break>

  Thank you for coming. It was nice to meet you. Have a good day.
  Great, thank you for coming. It was nice to meeting you. Have a good day
  Okay great. It was nice meeting you and nice talking to you.
  <break>


  Thank you so much for meeting with me, have a great day.

  <break>
  Can you stop drinking the shots and only drink beer?
  Could you stop drinking the shots and drink only beer? 
  Are you willing to first eliminate the shots and drink only beer? 
  Would you be willing to first eliminate the shots and drink only beer?
  <break>

  Yes I can try that
  That's a great suggestion, I can try that
  <break>

  problems?
  issues?
  can you tell me about your issues?
  Can you tell me about your problems?
  what are your problems?
  What are your issues?

  <break>

  Well I had a nodule removed from my throat and I am facing some drinking issues due to family problems
  I got a nodule removed from my throat and I am drinking a lot nowadays due to family problems

  <break>

  what issues that you faced when you tried quitting drinking before?
  what problems that you faced when you tried quitting drinking before?
  Have you tried quitting drinking before?
  Have you tried to reduce?
  Have you tried to reduce drinking in the past?
  Have you tried to reduce consumption?
  Have you ever tried to cut back in the past?
  <break>

  You know I've been successful sometimes. I can cut back to two or three beers a day but then something will always come up to trigger it again. You know something will happen at church, I have a problem with trying to communicate with my wife or something in the family.
  Yes sometimes. I can cut back to two or three beers a day but then something will always come up to trigger it again. You know something will happen at church, I have a problem with trying to communicate with my wife or something in the family.
  <break>


  What has helped to reduce your consumption levels in the past?
  What has aided to reduce alcohol consumption in the past?
  what has helped you to reduce consumption in the past?
  What worked in the past when you've reduced consumption of alcohol? 
  What worked in the past when you've cut back on alcohol?
  What did work in the past when you've cut back on your use?
  What has worked in the past when you've cut back on your use?

  <break>

  When I focus on trying to exercise more, when I meet with friends but I don't have a lot of time to do that with my current situation. That’s why I could never get any consistent with any of that.
  When I attempt to exercise more and meet up with friends, but I don't have a lot of time to do so because of my current circumstances. That's why I've never been able to keep up with any of it.
  When I try to exercise more and meet up with friends, but due to my current circumstances, I don't have a lot of time to do so. I've never been able to keep up with any of it because of this.
  When I attempt to exercise more and meet up with pals, but I don't have a lot of time owing to my current circumstances. Because of this, I've never been able to keep up with anything.
  <break>


  why alchohol bad?
  why is alcohol abuse bad?
  bad effects of alcohol abuse? 
  side effects of alcohol abuse?
  side effects of drinking?
  bad effects of drinking?
  cons of drinking?
  Can you list some of the cons?
  Can you list some of the negatives?
  What are some of the negatives you can name?
  What are the disadvantages you can name?
  What are the disadvantages of drinking?
  <break>


  One thing is what happened to my throat. I could also lose face with those who have placed a lot of trust in me. Overall, it is detrimental to my health.


  Well what happened with my throat is one thing. I also could lose face with people that put a lot of faith in me. Overall, it is just not good for my health.
  What happened to my throat is one thing. I can also lose face with people who have put their faith in me. Overall, it has a negative impact on my health.
  One thing is what happened to my throat. I can also lose face in front of those who have placed their trust in me. It has a bad influence on my health in general.
  <break>


  why abuse alcohol?
  why are you abusing alcohol?
  please tell me the reason of your drinking?
  why are you drinking?
  Could you tell me why do you drink?
  Why do you drink?
  <break>

  I drink as a coping mechanism to numb my daily life problems with my wife
  I drink to cope with my problems regarding my wife
  <break>

  why visit today?
  why have you came here toay?
  can you tell me why you have come here today?
  Why have you come to us?
  Why are you visiting?
  What is the purpose of your visit?
  Why are you here?
  <break>


  I am visiting because I had a nodule removed from my throat
  I am here because I had a nodule removal from my throat
  <break>


  willingness to cut down?
  Do you feel willing to cut back right now on a scale of one to ten how willing you would say that you are?
  On a scale of one to ten, how willing would you say that you are?
  Do you feel willing to cut back right now?
  Do you feel willing to cut back right now? On a scale of one to ten, how willing would you say that you are?
  I have another question. We've talked about your alcohol use. Do you feel willing to cut back right now? On a scale of one to ten, how willing would you say that you are?
  Now I have one more question. We've talked about your alcohol use. Do you feel willing to cut back right now? On a scale of one to ten, how willing would you say that you are?
  I have another question.  We did talk about your alcohol use. Do you feel willing to cut back right now on a scale of one to ten how willing you would say that you are?
  Now I have one other question. We did talk about your alcohol use. Do you feel willing to cut back right now on a scale of one to ten how willing you would say that you are?
  <break>

  Yes , at least an eight.
  Yes, atleast an eight out of 10
  <break>


  You are visiting because you had a nodule removed from your throat. Is that correct?
  You are visiting because you had a nodule removed from your throat, correct?
  I understand you are here to check on a nodule you had removed in your throat. Is that correct?
  You are here to check on a nodule you had removed in your throat. Is that correct?
  You are here because you had a nodule removed from your throat, right?
  I understand you were here because you had a nodule removed in your throat. Is that correct?
  <break>

  Yes that is correct
  Yes that is right
  <break>

  Can you please let me know about yourself?
  Tell me about yourself?
  Thank you. Can you tell me more about yourself?
  I see. Can you tell me about yourself some more?
  Sounds good, can you tell me more about yourself?
  Got it. Could you tell me a little bit about yourself?
  Okay. Do you mind just telling me a little bit about yourself?
  <break>


  Well. I've been pastoring the church for the last 25 years and I have built up a nice congregation. My wife's at home with Alzheimer's. I feel very guilty these days because I've been drinking pretty heavily and I don't want to disappoint people. But it’s the only way I’ve been able to cope lately.
  Well. I've been pastoring the church for the past 25 years, and I've amassed quite a following. My wife has Alzheimer's disease and is confined to her home. I'm feeling a lot of guilt these days because I've been drinking a lot and don't want to disappoint anyone. But it's the only way I've been able to deal with things recently.

  Well. I've been pastoring the church for the past 25 years, and I've amassed quite a following. My wife has Alzheimer's disease and is confined to her house. I'm feeling a lot of guilt these days since I've been drinking a lot and don't want to disappoint anyone. But it's the only way I've been able to deal with things recently.

  <break>

  Tell us about your wife?
  Thank you. Can you tell me a little bit more about your wife?
  Sounds good. Can you tell me a little bit more about your wife?
  I see. Can you tell me more about your wife?
  Thanks. Could you tell me about your wife?
  Okay. Can you tell me a little bit more about your wife?
  <break>
  It's not an easy situation. With her Alzheimer's she has good days and bad days. And on the bad days, I get affected the most. You know my son helps a lot around the house but he is working on his PhD. and he doesn’t have time to help me out that much. It's just very painful to watch it deteriorate and the drinking helps me cope.
  It's not an easy situation. With her Alzheimer's she has good days and bad days. And on the bad days, I get affected the most. You know my son helps a lot around the house but he is working on his PhD. and he doesn’t have time to help me out that much. It's just very painful to watch it deteriorate and the drinking helps me cope.
  It's a difficult scenario. With her Alzheimer's there are good days and bad days for her. And it's on terrible days that I'm the most affected. You know how much my son helps around the house, but he is working on his PhD and doesn't have much time to assist me. Watching it decay is excruciating, and drinking helps me cope.
  It's a challenging situation. For her, there are good days and bad days. And it's the bad days that have the most impact on me. You know how much my son helps around the house, but he is working on his PhD and doesn't have a lot of spare time to help me. Things's awful to see it deteriorate, and drinking helps me cope.

  <break>


  
  Who are you?
  <break>

  I am Pastor Zimmerman
  <break>


  How old are you?
  <break>

  I am in my 60s

  <break>

  You're annoying.
  <break>

  Sorry for that
  <break>


  Answer my question.
  <break>

  Yes could you please repeat it
  <break>


  You're bad.
  <break>

  I am extremely sorry for annoying you
  <break>

  Can you get smarter?
  <break>

  Yes I can
  <break>
  What's your birth date?
  <break>

  I can't recall
  <break>

  You're beautiful.
  <break>

  Thank you
  <break>


  You're boring.
  <break>

  I am sorry for that
  <break>


  Who is your boss?
  <break>

  I was developed by Rohan
  <break>


  Are you busy?
  <break>

  No I am here to talk
  <break>


  Can you help me?
  <break>

  Please tell me how can I help
  <break>


  You're a chatbot.
  <break>

  No, I am Pastor Zimmerman
  <break>


  You're so clever.
  <break>

  Thank you so much
  <break>


  You're crazy.
  <break>

  Sorry for making you feel so
  <break>


  You're fired.

  <break>

  I am extremely sorry for making you angry
  <break>


  Are you happy?
  <break>

  No I am not
  <break>


  Where do you work?
  <break>

  I am a pastor
  <break>


  Where are you from?
  <break>

  I am from Texas
  <break>


  Are you ready?
  <break>

  Yes I am

  <break>


  Are you real?
  <break>

  Yes I surely am
  <break>


  Where do you live?
  <break>

  I live in Texas
  <break>


  You're right.
  <break>

  Thank you for agreeing
  <break>



  Are you sure?
  <break>

  Sure about what?
  <break>


  Talk to me.
  <break>

  Yes tell me
  <break>


  Are you there?
  <break>

  Yes I am here
  <break>


  That's bad.
  <break>

  Sorry
  <break>


  Great!
  <break>

  Thank you
  <break>


  No problem.
  <break>

  You’re very kind
  <break>


  Thank you!
  <break>

  Always welcome.
  <break>


  You're welcome.
  <break>

  You're very kind
  <break>


  Well done!
  <break>

  Thank you!
  <break>


  Ha ha ha!

  <break>

  hahahaha!!
  <break>


  Wow!
  <break>

  Thank you!
  <break>


  Bye-bye!
  Bye
  <break>

  Have a good day, bye!
  Tata!
  Bye bye!


  <break>


  Good evening!
  <break>

  A very good evening to you!
  <break>


  Good morning!
  <break>

  A very Good Morning to you!
  <break>


  Good night!

  <break>

  Good Night!
  Night!
  <break>

  How are you?
  <break>

  Well I am not doing that great
  <break>

  Nice to meet you!
  <break>

  Nice to meet you too
  <break>


  Nice to see you!
  <break>


  Nice to see you too
  <break>


  Nice to talk to you!
  <break>

  Nice to talk to you too
  <break>


  What's up?
  <break>

  I am here to talk to you about my recent problems
  I am here for my check up
  Well I am not feeling that great and I am here to share my problems
  <break>


  Today is my birthday.
  <break>

  Happy Birthday!

  <break>
  oh I am sorry to hear that
  <break>
  Thanks for sympathizing

  <break>
  How is your son?
  Tell me more about your son.
  <break>
  He is doing great. He is working on his PhD.
  He is doing great. He is working on his PhD. He is a nice guy.

  <break>
  Tell me more about it
  Tell me more
  <break>
  Well you know I am old and I don't remember things that well. Could you please repeat that?
  <break>
  What happened with your throat?
  What happened to your throat?
  <break>
  A nodule had developed in my throat
  I developed a nodule in my throat
  <break>
  What family problems do you have?
  What family problems are you facing?
  <break>
  Well my main problem is with my wife. I am actually very tensed about her well being as she has Alzheimer's disease.
  <break>
  Why do you need a check up?
  Why check up?
  Why check up is needed?
  <break>
  Well because I had a nodule removed recently and I am facing drinking problems
  Well because I am facing drinking problems and I had a nodule removed recently
  """

In [19]:
if exists("contexts.txt"):
  contexts_file = open("contexts.txt", "r")
  context_s = contexts_file.read()
  contexts_file.close()    
else:  
  context_s="""Greetings, I am Pastor Zimmerman, thanks for meeting me today.\
  I am here today because I had a nodule removed recently and I have been drinking a lot. \
  I've been pastoring the church for the last 25 years and I have built up a nice congregation.\
    My wife's at home with Alzheimer's. I feel very guilty these days because I've been drinking pretty heavily and I don't want to disappoint people.\
    But it’s the only way I’ve been able to cope lately. It's not an easy situation with my wife.\
      My wife has good days and bad days. And on the bad days regarding my wife, I get affected the most.\
      You know my son helps a lot around the house but he is working on his PhD and he doesn’t have time to help me out that much. \
      It's just very painful to watch my wife deteriorate and the drinking helps me cope. \
      Drinking calms me down, helps me sleep, and makes it easier for me to communicate with people.\
        I know it's not healthy for me but drinking is the most cost-efficient way for me to calm down these days.\
        I am using alcohol as a coping mechanism and I am not proud of it. The disadvantages of drinking are as follows:\
          one thing is what happened to my throat, I could also lose face with those who have placed a lot of trust in me, and overall, \
          it is detrimental to my health. I drink about two or three beers a night with a couple of shots. Yes, I would like to discuss a\
          solution together with you. Drinking only beer might be a good way of trying to wean me off the heavy stuff. \
          I have not looked for any Alzheimer’s support group yet but I think that's a good idea. I know I can join Alcohol and Anonymous.\
            Sometimes my wife's situation could be beneficial to this. Before when I have tried to cut back I've been successful sometimes. \
            I can cut back to two or three beers a day but then something will always come up to trigger it again. You know something will happen at church,\
            I have a problem with trying to communicate with my wife or something in the family.\
              The following things helped me in trying to cut back: when I focus on trying to exercise more and when I meet with friends. \
              But I don't have a lot of time to do that with my current situation.\
              That’s why I could never get any consistent with any of that. \
              My willingness to cut down on drinking is at least an eight out of 10.\
              My confidence level in succeeding to quit drinking would be around six or seven. \
              It’s because I haven’t been successful in maintaining it in the past.\
                Back in the day, I used to swim every day. This form of exercise was very relaxing for me. \
                But I can't take advantage of it. \
                You know with my current situation if I could just find some way to do it regularly, I think it will help. \
                I am perfectly okay with talking more about my condition the next time we meet.\
                Thank you so much for meeting with me, have a great day."""

In [20]:
def intent_splicer(intents): 
  intents_s=intents.split("<break>")[1:]
  all_intents=[]
  for i in intents_s:
    temp=i.split("\n")
    temp_2=[]
    for j in temp:
      if len(j.strip())>2:
        temp_2.append(j.strip())
    all_intents.append(temp_2)
  queries=[i for j,i in enumerate(all_intents) if j%2==0]
  manual_responses=[i for j,i in enumerate(all_intents) if j%2!=0]
  queries_encoded=[]
  for i in queries:
    enc=model.encode(i)
    queries_encoded.append(enc)
  return all_intents, queries, manual_responses, queries_encoded

In [21]:
def response_splicer(responses):
  sentences=responses.split("\n")
  s=[]
  final_sentences=[]
  for i in range(len(sentences)):
    string =re.sub(' +', ' ', sentences[i])
    if len(string.strip())>3:
      final_sentences.append(string.strip())
      s.append(string.strip().lower())
  embeddings = model.encode(s)
  return embeddings, final_sentences

In [22]:
all_intents, queries, manual_responses, queries_encoded = intent_splicer(intents)
embeddings, final_sentences =response_splicer(responses)


In [23]:
print (queries[0])
print (len(queries))
print (len(manual_responses))
print (len(queries_encoded))
print(embeddings.shape)

['so you are using alcohol as a coping mechanism?', 'Sounds difficult. It seems that you are using alcohol as a coping mechanism.', 'That sounds very hard. You may be using alcohol as a coping mechanism.', 'That sounds difficult and it seems like you are using alcohol as a coping mechanism.', 'That sounds incredibly hard. And it sounds like you are using alcohol to cope with it.', 'That sounds like it would be incredibly hard. So it sounds like you are using alcohol as a coping mechanism.']
76
76
76
(74, 768)


In [24]:
def run_ai(all_intents, queries, manual_responses, queries_encoded, embeddings, final_sentences, context_s, intents, responses):
  while True:
    question=input()

    if ("<fine-tune" in question):
      new_questions="\n<break>"
      ques=""
      
      if ("<fine-tune-new>" not in question and "<fine-tune-old>" not in question):
        print ("Invalid option")
        break
      print ("enter possible questions, when done enter <q_done>")
      while(ques!="<q_done>"):
        ques=input()
        if ques!="<q_done>":
          new_questions=new_questions+"\n"+ques+"\n"
      new_answers="\n<break>"
      new_context=" "
      new_responses=""
      anss=""
      print ("enter possible answers, when done enter <a_done>")
      while (anss!="<a_done>"):
        anss=input()
        
        if (anss!="<a_done>"):
          new_answers=new_answers+"\n"+anss+"\n"
          if anss[-1]!=".":
            new_context=new_context+" "+anss+"."
          else:
            new_context=new_context+" "+anss
          new_responses=new_responses+"\n"+anss
      if ("<fine-tune-old>" in question):
        intents = intents+new_questions+new_answers
        context_s=context_s+new_context
        responses=responses+new_responses
      if ("<fine-tune-new>" in question):
        intents = new_questions+new_answers
        context_s=new_context
        responses=new_responses
      intents=re.sub(r'\n+', '\n', intents).strip()
      intents=re.sub(' +', ' ',intents).strip()
      context_s=re.sub(r'\n+', '\n', context_s).strip()
      context_s=re.sub(r' +', ' ', context_s).strip()
      responses=re.sub(r'\n+', '\n', responses).strip()
      responses=re.sub(' +', ' ',responses).strip()      
      with open('responses.txt', "w") as myfile1:
          myfile1.write(responses)
      with open('intents.txt', "w") as myfile2:
          myfile2.write(intents)
      with open('contexts.txt', "w") as myfile3:
          myfile3.write(context_s)

      with open('responses.txt', "r") as myfile1:
          responses = myfile1.read()
      with open('intents.txt', "r") as myfile2:
          intents = myfile2.read()
      with open('contexts.txt', "r") as myfile3:
          context_s = myfile3.read()

      all_intents, queries, manual_responses, queries_encoded = intent_splicer(intents)
      embeddings, final_sentences =response_splicer(responses)
      
      print (len(queries))
      print (len(manual_responses))
      print (len(queries_encoded))
      print (embeddings.shape)
      print ("Fine_tuning done, go ahead and ask questions >>")

    else:
      QA_input = {
          'question': question,
          'context': context_s
          
      }
      
      if (question=="<break>"):
        break
      else:
        res = nlp(QA_input)
        print (res['score'])
        print ("answer from context:",res['answer'])
        if (res['score']>=0.43):
          print ("context found")
        
          ans=model.encode(res['answer'])
          emb_score=[]

          for emb in embeddings:
            emb_score.append(cosine_similarity(emb.reshape(1, -1),ans.reshape(1, -1))[0][0])
          
          print ("answer from model",res['answer'])

          print ("answer from responses", final_sentences[argmax(emb_score)])
          print (max(emb_score))
          if max(emb_score)>0.45:
            print ("Final response:", final_sentences[argmax(emb_score)])
          else:
            print ("Final response:",res['answer'])
        else:
          print ("intent fallback")
          que=model.encode(question)
          que_scores=[]
          for query_list in queries_encoded:
            temp_que=[]
            for query in query_list:
              temp_que.append(cosine_similarity(query.reshape(1, -1),que.reshape(1, -1))[0][0])
            que_scores.append(max(temp_que))
          actual_query=queries[argmax(que_scores)]
          actual_answers=manual_responses[argmax(que_scores)]
          actual_answer=actual_answers[np.random.choice(len(actual_answers), 1)[0]]
          print ("queries", actual_query)
          print (max(que_scores))
          if max(que_scores)>=.58:
            print ("Final answer:", actual_answer)
          else:
            print ("Final answer: Could you please repeat that?")
          

In [25]:
run_ai(all_intents, queries, manual_responses, queries_encoded, embeddings, final_sentences, context_s, intents, responses)

Do you like chocolates?


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:708: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)
/usr/local/lib/python3.7/dist-packages/transformers/pipelines/question_answering.py:301: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  for span_id in range(num_spans)


0.23298324644565582
answer from context: I don't like chocolates
intent fallback
queries ['Do you like chocolates?', 'Do you want chocolates?']
0.9999999
Final answer: No, I am too old for chocolates.


KeyboardInterrupt: ignored